# Батарейка дня subprocess
Запуск процессов ОС и коммуникация с ними

Примеры работают на python-3.5

см [докуменацию](https://docs.python.org/3.5/library/subprocess.html?highlight=subprocess#module-subprocess)

`pydoc3 subprocess`

    Security
    --------
    Unlike some other popen functions, this implementation will never call
    /bin/sh implicitly.  This means that all characters, including shell
    metacharacters, can safely be passed to child processes.

### Как еще можно запускать программы?
```
import os
assert os.system('whoami') == 0

pid = os.spawnlp(os.P_NOWAIT, "/bin/mycmd", "mycmd", "myarg")
```

In [ ]:
import subprocess
print(subprocess.Popen.__doc__)
subprocess.Popen('true')

In [ ]:
sub = _
assert sub.returncode is None
pid = sub.pid
pid

In [ ]:
print('ps x -p %d' % pid)

In [ ]:
code = sub.poll()  # try now
assert sub.returncode == code
code

In [ ]:
sub = subprocess.Popen('sleep 1')

In [ ]:
sub = subprocess.Popen('sleep 1', shell=True)
sub.pid

In [ ]:
import time

start = time.time()
sub = subprocess.Popen(['sleep', '4'])

print('Checking for a child %d' % sub.pid)
assert sub.poll() is None

for i in range(4):
    print('Waiting a bit')
    time.sleep(0.5)
    assert sub.poll() is None

print('Waiting for a child %d' % sub.pid)
sub.wait()

print('Took %1.3f' % (time.time() - start))

# Мне нужен результат этой работы

In [ ]:
sub = subprocess.Popen('date')
assert sub.stdout is None

In [ ]:
print(sub.communicate.__doc__)
sub.communicate()

In [ ]:
sub = subprocess.Popen('date', stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
out, err = sub.communicate()
assert err is None
out

In [ ]:
out.decode('utf')

In [ ]:
sub = subprocess.Popen('which', stdout=subprocess.PIPE, stderr=subprocess.PIPE)
out, err = sub.communicate()
assert out == b''
err

In [ ]:
sub = subprocess.Popen('which', stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
out, err = sub.communicate()
assert err is None
out

In [ ]:
cmd = 'ps ax -U nesusvet'.split()
sub = subprocess.Popen(cmd, stdout=subprocess.PIPE)
out, err = sub.communicate()
out.split(b'\n')

In [ ]:
print(subprocess.check_output.__doc__)

In [ ]:
cmd = 'ps aux'.split()
out = subprocess.check_output(cmd)  # По-проще
out.split(b'\n')

# Аналог ps aux | grep python

In [ ]:
ps_aux = subprocess.Popen(
    'ps aux'.split(),
    stdout=subprocess.PIPE,
)
grep = subprocess.Popen(
    'grep python'.split(),
    stdin=ps_aux.stdout,
    stdout=subprocess.PIPE,
)
out, err = grep.communicate()
out.split(b'\n')

In [ ]:
import os
def show_tree():
    pid = os.getpid()
    cmd = 'pstree', '-p', str(pid)
    text = subprocess.check_output(cmd)
    print(text.decode('utf'))

show_tree()

In [ ]:
print(os.kill.__doc__)
print(os.waitpid.__doc__)

In [ ]:
os.wait()

In [ ]:
show_tree()

In [ ]:
print(subprocess.call.__doc__)
try:
    print('Run with timeout')
    subprocess.call(['sleep', '5'], timeout=3)
except subprocess.TimeoutExpired as ex:
    print('An exception occured %s' % ex)

In [ ]:
print(subprocess.check_call.__doc__)
subprocess.check_call(['which', 'python'])

In [ ]:
subprocess.check_call(['which', 'flynn'])

# Python вызывает другой python-процесс :)

In [ ]:
text = '''
import time

def main():
    time.sleep(5)

if __name__ == '__main__':
    print('Waiting now...')
    main()
'''
with open('tmp.py', 'w') as file_output:
    file_output.write(text)

In [ ]:
factory = (subprocess.Popen('python tmp.py'.split()) for _ in range(10))
for child in factory:
    print('New born child pid = %d' % child.pid)

print('Process tree')
show_tree()

print('Waiting 10 sec')
time.sleep(10)

show_tree()
print('Where are my children?')

In [ ]:
os.wait()

In [ ]:
show_tree()

# Применение
Я, как разработчик проекта, хочу помнить и делать как можно меньше ручных действий. Например, при изменении требований к параметрам запросов к API и изменении формата ответа, нужно регулярно обновлять документацию, что сводится к вызову `make`-задачи. Почему бы не использовать механизм хуков, существующий в `git`?

In [ ]:
#!/usr/bin/env python

import re
import subprocess

RE_RAW_DOC = re.compile(r'\.(json|raml)$')
RE_COMPILED_DOC = re.compile(r'\.(md)$')

GIT_LS_FILES = 'git diff --name-only --cached'.split()
GIT_ADD = 'git add -u'.split()

REBUILD_DOCS = 'make generate_docs'.split()


def iter_modified():
    output = subprocess.check_output(GIT_LS_FILES).decode('utf')
    for line in output.split('\n'):
        yield line


def docs_need_rebuild():
    for filename in iter_modified():
        if RE_RAW_DOC.search(filename):
            yield filename


def docs_was_rebuilt():
    for filename in iter_modified():
        if RE_COMPILED_DOC.search(filename):
            yield filename


def main():
    modified = list(docs_need_rebuild())
    rebuilt_docs = list(docs_was_rebuilt())
    if modified and not rebuilt_docs:
        subprocess.check_call(REBUILD_DOCS)

        rebuilt_docs = list(docs_was_rebuilt())
        subprocess.check_call(GIT_ADD + rebuilt_docs)


if __name__ == '__main__':
    main()

# Напоследок
### python-3.5
`subprocess.run() => subprocess.CompletedProcess`

# Дальше и глубже
- pipes
- multiprocessing